# Estimaciones en una encuesta


In [1]:
# ---------- # Importación de bibliotecas # ------------------ #
import os
import sys
import pandas as pd

# ---------- # Importación de funciones # ------------------ #
from modulos.func_analisis import *
from modulos.func_transformacion import *

Dentro del script "func_analisis" se encuentran las funciones "proporciones", "proporciones_des", "prevalencias", "prevalencias_des" y "tabulados_prevalencias". Todas estas funciones se pueden utilizar para generar estimaciones a partir de los datos de cualquier encuesta con un muestreo probabilístico estratificado. 

A grandes razgos las funciones requieren como argumentos el nombre de las varibles donde se almacena la información del ponderador, la unidad primaria de muestreo, el estrato, las preguntas que se desean analizar y el dataframe donde están dichas variables. 

Para usarlas, sólo es necesario definir correctamente sus argumentos. Los pasos que se recomienda seguir son:
1. Identificar las variables de interés en la base de datos asociada a la encuesta.
2. Homologar el tipo de datos de las variables de interés.
3. Estructurar un dataframe con las variables de interés.
4. Generar la estimación a partir de las respuestas a las preguntas seleccionadas.

A continuación se ejemplifica cómo se pueden generar estimaciones de alguna pregunta de interés con los datos de la Encuesta Nacional de Consumo de Drogas, Alcohol y Tabaco, ENCODAT 2016-2017.

In [2]:
# ------------------ # Carga de las bases de datos # ------------------ #
h=pd.read_csv(os.path.join('datos','originales','ENCODAT_2016_2017_Hogar','ENCODAT_2016_2017_Hogar.csv'), sep=';', low_memory=False)
bb=pd.read_csv(os.path.join('datos','originales','ENCODAT_2016_2017_Individual','ENCODAT_2016_2017_Individual.csv'), sep=';', low_memory=False)

## 1. Identificación de variables
En los argumentos de las funciones para hacer estimaciones, es necesario indicar el nombre de las variables que almacenan el factor de expansión, la unidad primaria de muestreo, el estrato y las preguntas de interés. Por ello, el primer paso es identificar si se tienen dichas variables y cuál es su nombre o clave.

En el caso de la ENCODAT:
- el factor de expansión está dentro de las variables 'ponde_ss' y 'ponde_hh'
- la unidad primaria de muestreo es la variable 'code_upm'
- el estrato están en la variable 'estrato'

Las preguntas de interés dependerán de la información que se desee estimar. 

Para este ejemplo, se eligieron dos preguntas:
- ¿Actualmente estudia? (clave ds8)
    1) No, nunca ha asistido a la escuela
    2) No, pero si fue a la escuela
    3) Sí
 
- ¿Cuál fue el último grado que ha completado o completó en la escuela? (clave ds9)
    1. Primaria incompleta (1 a 5 años)
    2. Primaria completa (6 años)
    3. Secundaria incompleta (1 a 2 años)
    4. Secundaria completa o equivalente (3 años)
    5. Bachillerato incompleto (1 a 2 años)
    6. Bachillerato completo o equivalente (aprox. 3 años)
    7. Estudios Universitarios incompletos (1 a 3 años)
    8. Estudios  Universitarios completos (4 a 5 años)
    9. Estudios de Posgrado (2 a 4 años)
    99) No contesta

In [3]:
# ---------- # Selección de preguntas # ------------------ #

#escribir aqui los códigos de las preguntas de interés
clv_preg = ['ds8','ds9']

## 2. Homologación de datos

Las variables de interés que requieren las funciones de "func_analisis" deben ser datos de tipo numérico, por ello para empezar a hacer estimaciones es necesario hacer una pequeña exploración a las bases de datos y en caso de ser necesario cambiar los datos, codificarlos o corregirlos para que sean de tipo numérico.

Siguiendo con el ejemplo de la ENCODAT, en la siguientes celdas se hace una pequeña exploración a las bases de datos y se homologan los datos a formato numérico.

### 2.1 Preparativos de las variables asociadas a la muestra

In [4]:
h[['estrato','code_upm','ponde_hh']].head()

,estrato,code_upm,ponde_hh
0,3,01_12,"170,475477457288"
1,3,01_12,"170,475477457288"
2,3,01_12,"170,475477457288"
3,3,01_12,"170,475477457288"
4,3,01_12,"170,475477457288"


In [5]:
h[['estrato','code_upm','ponde_hh']].dtypes

estrato      int64
code_upm    object
ponde_hh    object
dtype: object

Con las dos celdas anteriores se puede ver que la variable estrato ya tiene formato numérico, así que no vamos a hacerle ningún procesamiento. Sin embargo, las variables 'code_upm' y 'ponde_hh' sí necesitan homologarse a formato numérico, además la variable "ponde_hh" tiene más información que sólo el ponderador.

En la siguiente celda se hacen las adecuaciones correspondientes para solucionar los problemas anteriores:

In [6]:
# Homologación de codigos de las unidades primarias de muestreo para que sean sólo numéricos
h.insert(0,'upm',h['code_upm'].str.replace('_','').astype(int))

# Ahora bien, en el caso muy específico de esta base de datos,
#el factor de expansión son los primeros dígitos separados por la coma en la variable 'ponde_hh'
#así que se genera una nueva variable sólo con el ponderador en formato numérico
h.insert(0,'factor_exp',h['ponde_hh'].str.partition(',')[0].astype(int))

De igual forma, en la base 'bb', la variable "ponde_ss" tiene más información que sólo el ponderador, así que se corrige:

In [7]:
bb['ponde_ss'].head()

0    263,004588042305
1    779,543148763914
2    263,004588042305
3     281,00343231265
4    421,505148468975
Name: ponde_ss, dtype: object

In [8]:
# Se genera una nueva variable sólo con el ponderador en formato numérico
bb.insert(0,'factor_exp',bb['ponde_ss'].str.partition(',')[0].astype(int))

### 2.2 Preparativos de las variables asociadas a las preguntas
En el caso de la ENCODAT las respuestas a preguntas se codifican mediante claves, sin embargo no siempre están homologadas para que efectivamente sean sólo datos numéricos. En la siguiente celda podemos ver como una de las preguntas seleccionadas ('ds9') tiene datos de distintos tipos, no sólo numéricos.

In [10]:
# ---------- # Revisión del tipo de respuestas de las preguntas seleccionadas # ------------------ #
bb[clv_preg].dtypes

ds8     int64
ds9    object
dtype: object

In [11]:
bb.ds9.unique()

array(['4', '6', '5', '1', '3', '2', '7', '9', '8', ' ', '99'],
      dtype=object)

En una inspección más detallada (en la celda anterior) podemos ver que hay dos problemas con el tipo de datos almacenados en la variable 'ds9':
- el primero es que hay espacios vacíos
- el segundo es que todas las claves son cadenas de caracteres.

Para solucionar ambos problemas se hace lo siguiente:

In [12]:
# ---------- # Homologación de los datos de las preguntas seleccionadas # ------------------ #
#pregunta con problemas
clave_pregunta = 'ds9'

#para solucionar el primer problema es necesario asignar una clave numérica a los datos vacíos
#en este caso se elige asignar el 0 para los espacios vacíos
bb[clave_pregunta]=bb[clave_pregunta].replace(' ',0)

#para solucionar el segundo problema se cambia el tipo de dato, de cadena de caracteres a numérico
bb[clave_pregunta]=bb[clave_pregunta].astype(int)

NOTA: la homologación y estructuración de una base de datos es de las partes más complejas del análisis de datos porque los datos pueden tener infinidad de variaciones o problemas, cada uno con su solución particular.

Sin embargo, una forma general de abordar esto es inspeccionar las respuestas y homologar la forma en la que se escriben dichas respuestas. 

## 3. Estructuración del dataframe

La información de la ENCODAT está distribuida en dos bases de datos (nombradas en este cuaderno como "bb" y "h"), por ello en este caso la estructuración del dataframe requiere unir dichas bases. 

NOTA: Cada base puede necesitar sus adecuaciones particulares para estructurar el Dataframe que requieren las funciones del script "func_analisis".



In [13]:
# ---------- # Creación de dataframe para funciones # ------------------ #

# Creación de variable 'id_hogar' para unir la base "bb" con la base "h"
bb.insert(0,'id_hogar',bb['id_pers'].str[:20])

# Unión de las bases en un solo dataframe mediante el id_hogar
b=bb.merge(h[['id_hogar','estrato','upm','code_upm','est_var']],on='id_hogar',how='left')

## 4. Estimación

### 4.1 ¿Con qué proporción se mencionó cada respuesta de la pregunta seleccionada?
Para saber con que proporción se mencionó cada respuesta de las preguntas seleccionadas se puede utilizar la función "proporciones". Para utilizarla, basta llamar a la función y dar los siguientes argumentos:
- b) dataframe con la base de datos.
- clave_preg: cadena de caracteres con la clave de la pregunta para la que se desea estimar las proporciones.
- alp: valor entre 0 y 1 para definir el nivel de significancia del intervalo (el valor predeterminado es 0.95).
- ponderador: cadena de caracteres con el nombre de la variable con el ponderador de la encuesta (el valor predeterminado es "factor_exp").
- estrato: cadena de caracteres con el nombre de la variable con los estratos de la encuesta (el valor predeterminado es "estrato").
- upm: cadena de caracteres con el nombre de la columna con las unidades primarias de la encuesta (el valor predeterminado es "upm").

Al ejecutar la celda, la función genera estimaciones puntuales, su respectivo intervalo de confianza, el coeficiente de variación y el error estándar:

In [14]:
proporciones(b = b, 
             clave_preg = 'ds9', 
             alp = 0.95, 
             ponderador = 'factor_exp', 
             estrato= 'estrato', 
             upm = 'upm')

,clave_respuesta,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
0,0,2.235656,1.998388,2.472924,1905531,0.121057,5.414847
1,1,7.543581,7.150328,7.936835,6429670,0.200643,2.659790
2,2,15.188253,14.563453,15.813053,12945503,0.318781,2.098869
3,3,10.088452,9.668178,10.508726,8598756,0.214429,2.125493
4,4,25.132433,24.390835,25.874031,21421291,0.378373,1.505518
5,5,8.865000,8.431773,9.298226,7555963,0.221038,2.493378
6,6,16.721276,15.973866,17.468685,14252154,0.381339,2.280559
7,7,5.092753,4.575115,5.610390,4340739,0.264106,5.185910
8,8,8.088889,7.508010,8.669769,6894456,0.296373,3.663948
9,9,0.884170,0.742965,1.025374,753610,0.072045,8.148282


La columna 'estimación' del dataframe que genera la función 'proporciones' describe con que proporción la población (estimada por la encuesta) eligió las respuestas.

Por ejemplo, con la ENCODAT, se estima que el 15.18% de la población tiene la primaria completa (respuesta que corresponde a la clave 2)

### 4.2 ¿Con qué proporción se mencionó cada respuesta de la pregunta seleccionada? 
### Desagregado por otra variable

In [15]:
proporciones_des(b = b,
                 clave_preg = 'ds8',
                 var_des = 'ds9',
                 alp = 0.95,
                 ponderador = 'factor_exp',
                 estrato= 'estrato',
                 upm = 'upm')

NameError: name 'monitoreo' is not defined

In [ ]:
# ---------- # Adecuaciones a las bases de datos # ------------------ #

# Creación de grupos etarios 
#(los grupos etarios se pueden cambiar si así se desea) 
grupos_etarios=['12-17', '18-34', '35-59', '60-75']
divisiones=[int(x[:2])-1 for x in grupos_etarios]+[int(grupos_etarios[-1][-2:])]
b.insert(len(b.columns),'grupo_etario',pd.cut(list(b['ds3']), bins=divisiones, labels=grupos_etarios))

# Cambio de nombre y valores de las columnas entidad y sexo
b.rename(columns={'entidad':'cve_ent','desc_ent':'nom_ent','ds2':'sexo'}, inplace=True)
valores_sexo = {1:'Hombre', 2:'Mujer'}
b['sexo'] = b['sexo'].map(valores_sexo).fillna(b['sexo'])
b['nom_ent']=b['nom_ent'].str.strip().apply(arreglar_palabras)